In [1]:
# Todo read about tf print, print and graphs. Check the differences. How tf2 creates graphs
import logging
import tensorflow as tf
from tensorflow import keras
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import argparse 

logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)
log = logging.getLogger()

%config Completer.use_jedi = False # make autocompletion works in jupyter

ERROR! Session/line number was not unique in database. History logging moved to new session 149


In [2]:
import argparse 

args = argparse.Namespace()
args.data_folder = './data/bach-next-note/'
args.train_folder = args.data_folder + 'train/'
args.val_folder = args.data_folder + 'valid/'
args.test_folder = args.data_folder + 'test/'
# args.train_fraction = 0.8
args.seed = 101
args.batch_size = 32
args.epochs = 7

In [3]:
import matplotlib.pyplot as plt
from pathlib import Path

def plot_history(history):
    log.info("History keys: %s", history.history.keys())
    # Accuracy
    fig, ax = plt.subplots(figsize=(10, 5))
    ax.plot(history.history['accuracy'], label='Train')
    ax.plot(history.history['val_accuracy'], label='Test')
    ax.set_title('Model accuracy')
    ax.set_ylabel('Accuracy')
    ax.set_xlabel('Epoch')
    ax.grid(True)
    ax.legend(['Train', 'Val'], loc='lower right')
    
    # Loss
    plt.figure(figsize=(10,5))
    plt.plot(history.history['loss'])
    plt.plot(history.history['val_loss'])
    plt.title('Model loss')
    plt.ylabel('Loss')
    plt.xlabel('Epoch')
    plt.legend(['Train', 'Val'], loc='upper left')
    plt.grid()
    plt.show()
    
# couldn't make it work
# def build_dataset(data_folder):
#     def preprocess(line):
#         csv_records = tf.io.decode_csv(line, record_defaults=[tf.constant(0)]*4)
#         print(csv_records)
#         x = tf.stack(csv_records)
#         return x
#     dataset = tf.data.Dataset.list_files(data_folder + 'chorale_*.csv', seed=args.seed)
#     dataset = dataset.interleave(lambda filepath: tf.data.TextLineDataset(filepath).skip(1))
#     dataset = dataset.map(preprocess)
# #     dataset = dataset.shuffle(10000)
#     dataset = dataset.batch(args.batch_size).prefetch(1)
#     for d in dataset:
#         print(d)
#         break
#     return dataset

def build_dataset(data_folder):
    paths = Path(data_folder).glob('**/chorale_*.csv')
    np_list = [pd.read_csv(p).to_numpy() for p in paths]
    ragged = tf.ragged.constant(np_list)
    dataset = tf.data.Dataset.from_tensor_slices(ragged)
    def create_target(batch):
        print(batch)
        return batch
#     def batch_window(window):
#         return window.batch(32 + 1)
#     def to_windows(chorale):
#         dataset = tf.data.Dataset.from_tensor_slices(chorale)
#         dataset = dataset.window(32 + 1, 16, drop_remainder=True)
#         return dataset.flat_map(batch_window)
    dataset = dataset.flat_map(tf.data.Dataset.from_tensor_slices).flat_map(tf.data.Dataset.from_tensor_slices) # to a seq of notes
    dataset = dataset.batch(32)
    dataset = dataset.map(create_target)
    print(dataset)
    for d in dataset:
        print(d)
        break
#     def preprocess(line):
#         csv_records = tf.io.decode_csv(line, record_defaults=[tf.constant(0)]*4)
#         print(csv_records)
#         x = tf.stack(csv_records)
#         return x
#     dataset = tf.data.Dataset.list_files(data_folder + 'chorale_*.csv', seed=args.seed)
#     dataset = dataset.interleave(lambda filepath: tf.data.TextLineDataset(filepath).skip(1))
#     dataset = dataset.map(preprocess)
# #     dataset = dataset.shuffle(10000)
#     dataset = dataset.batch(args.batch_size).prefetch(1)

#     return dataset



def baseline_model():
    model = keras.models.Sequential()
    model.add(keras.layers.SimpleRNN(20, return_sequences=True, input_shape=[None, 4]))
    model.add(keras.layers.SimpleRNN(20))
    model.add(keras.layers.Dense(4))
    model.compile(loss='mse',optimizer=keras.optimizers.Nadam(learning_rate=0.01), metrics=[keras.metrics.mean_squared_error])
    return model

train_dataset = build_dataset(data_folder=args.train_folder)
# val_dataset = build_dataset(data_folder=args.val_folder)
# model = baseline_model()
# model.fit(train_dataset, epochs=1, validation_data=val_dataset)

Tensor("args_0:0", shape=(None,), dtype=int32)
<MapDataset shapes: (None,), types: tf.int32>
tf.Tensor(
[66 61 57 54 66 61 57 54 68 61 59 54 68 61 59 54 69 66 61 54 69 66 61 56
 69 66 61 57 69 66 61 59], shape=(32,), dtype=int32)


In [2]:
import logging
import tensorflow as tf
from tensorflow import keras
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import argparse 

logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)
log = logging.getLogger()

%config Completer.use_jedi = False # make autocompletion works in jupyter

import argparse 

args = argparse.Namespace()
args.data_folder = './data/bach-next-note/'
args.train_folder = args.data_folder + 'train/'
args.val_folder = args.data_folder + 'valid/'
args.test_folder = args.data_folder + 'test/'
# args.train_fraction = 0.8
args.seed = 101
args.batch_size = 32
args.epochs = 7

from pathlib import Path

def create_target(batch):
    print(batch)
    X = batch[:, :-1]
    Y = batch[:, 1:] # predict next note in each arpegio, at each step
    return X, Y

def preprocess(window):
    min_note = 36
    window = tf.where(window == 0, window, window - min_note + 1) # shift values
    return tf.reshape(window, [-1]) # convert to arpegio

def bach_dataset(chorales, batch_size=32, shuffle_buffer_size=None,
                 window_size=32, window_shift=16, cache=True):
    def batch_window(window):
        return window.batch(window_size + 1)

    def to_windows(chorale):
        dataset = tf.data.Dataset.from_tensor_slices(chorale)
        dataset = dataset.window(window_size + 1, window_shift, drop_remainder=True)
        dataset = dataset.flat_map(batch_window)
        return dataset

    chorales = tf.ragged.constant(chorales, ragged_rank=1)
#     print("chorale[0] {}".format(chorales[0]))
    dataset = tf.data.Dataset.from_tensor_slices(chorales)
    dataset = dataset.flat_map(to_windows)
    for d in dataset:
        tf.print(f"d after flat_map: {d}")
        break
    dataset = dataset.map(preprocess)
    for d in dataset:
        tf.print(f"d after preprocess: {d}")
        break
    
    if cache:
        dataset = dataset.cache()
    if shuffle_buffer_size:
        dataset = dataset.shuffle(shuffle_buffer_size)
    dataset = dataset.batch(batch_size)
    dataset = dataset.map(create_target)
    for d in dataset:
        print(f"final dataset: {d}")
        break
    return dataset.prefetch(1)

paths = Path(args.train_folder).glob('**/chorale_*.csv')
np_list = [pd.read_csv(p).values.tolist() for p in paths]
# print(len(np_list[0]))
# print(np_list[0])
bach_dataset(np_list)

d after flat_map: [[66 61 57 54]
 [66 61 57 54]
 [68 61 59 54]
 [68 61 59 54]
 [69 66 61 54]
 [69 66 61 56]
 [69 66 61 57]
 [69 66 61 59]
 [68 65 61 61]
 [68 65 61 61]
 [68 65 59 49]
 [68 65 59 49]
 [66 66 57 50]
 [66 66 57 50]
 [66 66 57 50]
 [66 66 57 50]
 [66 66 59 50]
 [66 66 59 50]
 [68 66 59 50]
 [68 66 59 50]
 [69 66 61 49]
 [69 66 61 49]
 [69 66 61 47]
 [69 66 61 47]
 [68 65 61 49]
 [68 65 61 49]
 [68 65 59 49]
 [68 65 59 49]
 [66 61 57 42]
 [66 61 57 42]
 [66 61 57 42]
 [66 61 57 42]
 [73 66 57 54]]
d after preprocess: [31 26 22 19 31 26 22 19 33 26 24 19 33 26 24 19 34 31 26 19 34 31 26 21
 34 31 26 22 34 31 26 24 33 30 26 26 33 30 26 26 33 30 24 14 33 30 24 14
 31 31 22 15 31 31 22 15 31 31 22 15 31 31 22 15 31 31 24 15 31 31 24 15
 33 31 24 15 33 31 24 15 34 31 26 14 34 31 26 14 34 31 26 12 34 31 26 12
 33 30 26 14 33 30 26 14 33 30 24 14 33 30 24 14 31 26 22  7 31 26 22  7
 31 26 22  7 31 26 22  7 38 31 22 19]
Tensor("args_0:0", shape=(None, None), dtype=int32)
final datas

<PrefetchDataset shapes: ((None, None), (None, None)), types: (tf.int32, tf.int32)>

In [3]:
train_set = bach_dataset(train_chorales, shuffle_buffer_size=1000)
valid_set = bach_dataset(valid_chorales)
test_set = bach_dataset(test_chorales)

NameError: name 'train_chorales' is not defined

In [53]:
tf.ragged.constant([[1, 2], [3], [4, 5, 6]], ragged_rank=1)
tf.ragged.constant([[[1, 2], [3, 2], [4, 5]], [[1, 2]]], ragged_rank=2)
# tf.ragged.constant([[[1], [2]], [[3], [4]], [[5], [6]]], ragged_rank=1)

<tf.RaggedTensor [[[1, 2], [3, 2], [4, 5]], [[1, 2]]]>

In [15]:
dataset = tf.data.Dataset.range(7).window(3, 2)
for d in dataset:
    print(d)
    print([item.numpy() for item in d])

<_VariantDataset shapes: (), types: tf.int64>
[0, 1, 2]
<_VariantDataset shapes: (), types: tf.int64>
[2, 3, 4]
<_VariantDataset shapes: (), types: tf.int64>
[4, 5, 6]
<_VariantDataset shapes: (), types: tf.int64>
[6]


In [55]:
dataset = tf.data.Dataset.from_tensor_slices([[1, 2], [3, 4], [5, 6], [7, 8]]).window(2, 1, drop_remainder=True)
for d in dataset:
    print(d)
    print([item.numpy() for item in d])
    
def b(window):
    return window.batch(3)

dataset = dataset.flat_map(b)
print("flat_map")
for d in dataset:
    print(d)
    print([item.numpy() for item in d])

def b1(window):
    window = tf.where(window == 1, window, window - 1) # shift values
    tf.print(window)
    return window

print("map1")
dataset = dataset.map(b1)
for d in dataset:
    print(d)
    print([item.numpy() for item in d])
    
def b2(window):
    window = tf.reshape(window, [-1])
    tf.print(window)
    return window

print("map2")
dataset = dataset.map(b2)
for d in dataset:
    print(d)
    print([item.numpy() for item in d])

<_VariantDataset shapes: (2,), types: tf.int32>
[array([1, 2], dtype=int32), array([3, 4], dtype=int32)]
<_VariantDataset shapes: (2,), types: tf.int32>
[array([3, 4], dtype=int32), array([5, 6], dtype=int32)]
<_VariantDataset shapes: (2,), types: tf.int32>
[array([5, 6], dtype=int32), array([7, 8], dtype=int32)]
flat_map
tf.Tensor(
[[1 2]
 [3 4]], shape=(2, 2), dtype=int32)
[array([1, 2], dtype=int32), array([3, 4], dtype=int32)]
tf.Tensor(
[[3 4]
 [5 6]], shape=(2, 2), dtype=int32)
[array([3, 4], dtype=int32), array([5, 6], dtype=int32)]
tf.Tensor(
[[5 6]
 [7 8]], shape=(2, 2), dtype=int32)
[array([5, 6], dtype=int32), array([7, 8], dtype=int32)]
map1
[[1 1]
 [2 3]]
tf.Tensor(
[[1 1]
 [2 3]], shape=(2, 2), dtype=int32)
[array([1, 1], dtype=int32), array([2, 3], dtype=int32)]
[[2 3]
 [4 5]]
tf.Tensor(
[[2 3]
 [4 5]], shape=(2, 2), dtype=int32)
[array([2, 3], dtype=int32), array([4, 5], dtype=int32)]
[[4 5]
 [6 7]]
tf.Tensor(
[[4 5]
 [6 7]], shape=(2, 2), dtype=int32)
[array([4, 5], dt